In [2]:
import pandas as pd
from memory_fix import reduce_mem_usage
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Read Data 

In [3]:
group = pd.read_csv('data/small_group_description.csv')

transactions_train = pd.read_csv('data/transactions_train.csv')
train_target = pd.read_csv('data/train_target.csv')

transactions_test = pd.read_csv('data/transactions_test.csv')
test_id = pd.read_csv('data/test.csv')

In [4]:
transactions_train = reduce_mem_usage(transactions_train, use_float16=True)
train_target = reduce_mem_usage(train_target, use_float16=True)
transactions_test = reduce_mem_usage(transactions_test, use_float16=True)

Memory usage of dataframe is 807.21 MB
Memory usage after optimization is: 302.70 MB
Decreased by 62.5%
Memory usage of dataframe is 0.46 MB
Memory usage after optimization is: 0.14 MB
Decreased by 68.7%
Memory usage of dataframe is 539.16 MB
Memory usage after optimization is: 202.19 MB
Decreased by 62.5%


## Get data stats

In [5]:
def get_target_stats():
    mean_dict = {}
    sum_dict = {}
    median_dict = {}
    max_dict = {}
    
    all_stat_data = transactions_train.groupby('client_id')['amount_rur'].agg(['mean', 'sum', 'median', 'max']).reset_index()
    
    
    stat_data = pd.merge(train_target, all_stat_data, on='client_id')
    
    for cls in range(0, 4):
        mean_stat = np.mean(stat_data[stat_data.bins == cls]['mean'])
        sum_stat = np.mean(stat_data[stat_data.bins == cls]['sum'])
        median_stat = np.mean(stat_data[stat_data.bins == cls])['median']
        max_stat = np.mean(stat_data[stat_data.bins == cls])['max']
        
        mean_dict[cls] = mean_stat
        sum_dict[cls] = sum_stat
        median_dict[cls] = median_stat
        max_dict[cls] = max_stat
        
    return mean_dict, sum_dict, median_dict, max_dict


In [6]:
mean_dict, sum_dict, median_dict, max_dict = get_target_stats()

In [7]:
def get_trendline_cf(data):
    # eeee govno-code пиздец
    coef_df = {}
    for ind in tqdm(set(data['client_id'])):
        user_money = np.array(data[data.client_id == ind]['amount_rur'])
        
        for i in range(1, len(user_money)):
            user_money[i] += user_money[i-1]
            
        a_coeff, _ = poly_trend(user_money)
        
        coef_df[ind] = a_coeff
    
    
    coef_df = pd.DataFrame(coef_df.values(), index=coef_df.keys(),
                           columns=['trend_coeff']).reset_index().rename(columns={'index':'client_id'})


    return coef_df

## Preproc functions 

In [8]:
def get_day_stats(data):
    counter_df = data.groupby(['client_id','trans_date'])['amount_rur'].sum()

    cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                              columns='trans_date', values='amount_rur').fillna(0)
    
    cat_counts.columns = [f'day_{i}_sum' for i in cat_counts.columns]
    
    
    return cat_counts

In [9]:
def get_max_trans_group(data, is_train=True):
    # get max trans group per day
    if is_train:
        counter_df = transactions_train.sort_values("amount_rur", ascending=False).groupby(['client_id','trans_date'], as_index=False).first()
    else:
        counter_df = transactions_test.sort_values("amount_rur", ascending=False).groupby(['client_id','trans_date'], as_index=False).first()

    cat_counts = counter_df.pivot(index='client_id', \
                                              columns='trans_date', values='small_group').fillna(0)
    
    cat_counts.columns = [f'max_trans_group_day_{i}' for i in cat_counts.columns]
    
    return cat_counts
    

In [10]:
def poly_trend(timeseries, degree=1):
    """
    Поиск прямой, которая максимально соответствует тренду TS
    Возвращает коэффициенты линейной функции вида f(x) = ax + b
    """
    X = np.arange(0, len(timeseries))
    Y = np.array(timeseries)
    z = np.polyfit(X, Y, degree)

    return z[0],z[1]

In [11]:
def max_trans_group(data):
    lst = data.groupby(['client_id']).max()['small_group']
    lst = pd.DataFrame(lst).rename(columns={'small_group': "max_trans_group"})
    return lst

In [12]:
def max_trans_day(data):
    lst = data.groupby(['client_id']).max()['trans_date']
    lst = pd.DataFrame(lst).rename(columns={'trans_date': "max_trans_date"})
    return lst

In [13]:
def get_stat(data, func='count'):
    if func == "count":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].count()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_'+str(i) for i in cat_counts.columns]
    
    elif func == "sum":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].sum()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_sum_'+str(i) for i in cat_counts.columns]
    
    elif func == 'std':
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].std()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_std_'+str(i) for i in cat_counts.columns]
        
    elif func == "median":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].median()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_median_'+str(i) for i in cat_counts.columns]
        
    return cat_counts


In [14]:
def preproc_data(data, is_train = True):
    agg_features = data.groupby('client_id')['amount_rur'].agg(['count', 'sum','mean','std','min','max', 'median']).reset_index()
    
    
    day_stats = get_day_stats(data)
    cat_counts = pd.concat([get_stat(data, func='count'), get_stat(data, func='sum'),
                            get_stat(data, func='std'), get_stat(data, func='median'),
                            day_stats], axis=1)



    if is_train:
        preproced = pd.merge(train_target, agg_features, on='client_id')
        max_trans_group_data = get_max_trans_group(data)
    else:
        preproced = pd.merge(test_id, agg_features, on='client_id')
        max_trans_group_data = get_max_trans_group(data, False)
        
    max_trans = max_trans_group(data)
    max_trans_date = max_trans_day(data)

    
    
    preproced = pd.merge(preproced, max_trans, on='client_id')
    preproced = pd.merge(preproced, max_trans_date, on='client_id')
    
    preproced = pd.merge(preproced, max_trans_group_data, on='client_id')


    
#     if is_train:
#         preproced = pd.merge(preproced, train_trendline, on='client_id')
#     else:
#         preproced = pd.merge(preproced, test_trendline, on='client_id')

    preproced = pd.merge(preproced, cat_counts.reset_index(), on='client_id')
    
    for y in range(0, 4):
        preproced[f'mean_deviation_cls_{y}'] = preproced['mean'].apply(lambda x: x - mean_dict[y])
#         preproced[f'sum_deviation_cls_{y}'] = preproced['sum'].apply(lambda x: x - sum_dict[y])
        preproced[f'meadian_deviation_cls_{y}'] = preproced['median'].apply(lambda x: x - median_dict[y])
#         preproced[f'max_deviation_cls_{y}'] = preproced['max'].apply(lambda x: x - max_dict[y])

    

#     preproced['range_value'] = preproced['max'] - preproced['min']
        
    return preproced


## Apply preprocessing 

to_s_train = get_trendline_cf(transactions_train)
to_s_train.to_csv('train_trendline.csv')

to_s_test = get_trendline_cf(transactions_test)
to_s_test.to_csv('test_trendline.csv')

In [26]:
train_trendline = pd.read_csv('train_trendline.csv').drop('Unnamed: 0', axis=1)
test_trendline = pd.read_csv('test_trendline.csv').drop('Unnamed: 0', axis=1)

In [28]:
train = preproc_data(transactions_train)
test = preproc_data(transactions_test, False)

In [29]:
common_features = list(set(train.columns).intersection(set(test.columns)))

In [30]:
train_target = train['bins']
train_data = train[common_features]
X_test = test[common_features]

In [31]:
train_data.shape

(30000, 2278)

## Params search

In [39]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
import lightgbm as lgb
import catboost as cb

In [35]:
! install bayesian-optimization

Traceback (most recent call last):
  File "/home/furfa/anaconda3/bin/pip", line 11, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable


In [49]:
X_train, X_test, y_train, y_test = train_test_split(train_data, np.array(train_target), test_size=0.33, random_state=42)

In [47]:
bounds = {
    'learning_rate': (0.001, 0.3),
    'num_leaves': (30, 500), 
    'bagging_fraction' : (0.1, 1),
    'feature_fraction' : (0.1, 1),
    'min_child_weight': (0.001, 0.5),   
    'min_data_in_leaf': (20, 240),
    'max_depth': (-1, 32),
    'reg_alpha': (0.1, 4), 
    'reg_lambda': (0.1, 10)
}

def train_model(learning_rate, 
                num_leaves,
                bagging_fraction, 
                feature_fraction, 
                min_child_weight,
                min_data_in_leaf,
                max_depth,
                reg_alpha,
                reg_lambda):
    
    params = {'learning_rate': learning_rate,
              'num_leaves': int(num_leaves), 
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
              'min_child_weight': min_child_weight,   
              'min_data_in_leaf': int(min_data_in_leaf),
              'max_depth': int(max_depth),
              'reg_alpha': reg_alpha, 
              'reg_lambda': reg_lambda,
              'objective': 'multiclass',
              'boosting_type': 'gbdt',
              'num_class':4,
              'random_state': 47,
              'verbosity': 0,
              'metric': 'multi_logloss',
              'n_estimators': 2000,
              'n_jobs': -1
                }
    
    trn_data = lgb.Dataset(X_train, y_train)
    val_data = lgb.Dataset(X_test, y_test)
    model = lgb.train(params, trn_data, 5000, valid_sets = [trn_data, val_data],
                      verbose_eval=0, early_stopping_rounds=50)
    
    return -model.best_score['valid_1']['multi_logloss']


In [50]:
optimizer = BayesianOptimization(f=train_model, pbounds=bounds, random_state=47)
optimizer.maximize(init_points=10, n_iter=10)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_da... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.8607   |  0.2021   |  0.977    |  0.2189   |  10.6     |  0.3541   |  195.9    |  333.4    |  1.717    |  7.09     |
|  2        | -0.8952   |  0.322    |  0.3304   |  0.008179 |  2.258    |  0.1509   |  161.0    |  181.4    |  0.8234   |  9.18     |
|  3        | -0.8425   |  0.3438   |  0.3462   |  0.2864   |  3.195    |  0.3739   |  21.15    |  432.7    |  2.814    |  5.575    |
|  4        | -0.853    |  0.9417   |  0.5614   |  0.05411  |  16.72    |  0.1474   |  22.33    |  445.4    |  2.66     |  9.428    |
|  5        | -0.8503   |  0.7705   |  0.3405   |  0.1092   |  16.37    |  0.2739   |  76.91    |  112.1    |  1.507    |  1.488    |
|  6        | -0.8418   |  0.4502   |  0.5247   |  0.2907   | 

In [53]:
optimizer.max

{'target': -0.8362082881806135,
 'params': {'bagging_fraction': 0.64183588863209,
  'feature_fraction': 0.8517663630293925,
  'learning_rate': 0.026527375714140905,
  'max_depth': 15.113597110603894,
  'min_child_weight': 0.45025619488575386,
  'min_data_in_leaf': 221.75229253610547,
  'num_leaves': 131.93525410172379,
  'reg_alpha': 1.6624228537423238,
  'reg_lambda': 5.4972043534631}}

In [78]:
best_lgb = {'bagging_fraction': 0.64183588863209,
  'feature_fraction': 0.8517663630293925,
  'learning_rate': 0.026527375714140905,
  'max_depth': 15.113597110603894,
  'min_child_weight': 0.45025619488575386,
  'min_data_in_leaf': 221.75229253610547,
  'num_leaves': 131.93525410172379,
  'reg_alpha': 1.6624228537423238,
  'reg_lambda': 5.4972043534631}


best_lgb = {'learning_rate': best_lgb['learning_rate'],
              'num_leaves': int(best_lgb['num_leaves']), 
              'bagging_fraction' : best_lgb['bagging_fraction'],
              'feature_fraction' : best_lgb['feature_fraction'],
              'min_child_weight': best_lgb['min_child_weight'],   
              'min_data_in_leaf': int(best_lgb['min_data_in_leaf']),
              'max_depth': int(best_lgb['max_depth']),
              'reg_alpha': best_lgb['reg_alpha'], 
              'reg_lambda': best_lgb['reg_lambda'],
              'objective': 'multiclass',
              'boosting_type': 'gbdt',
              'num_class':4,
              'random_state': 47,
              'verbosity': 0,
              'metric': 'multi_logloss',
              'n_estimators': 2000,
              'n_jobs': -1
                }

model_lgb = lgb.LGBMClassifier(**best_lgb)

In [81]:
res = cross_validate(model_lgb, train_data, np.array(train_target), cv=3, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 33.8min finished


In [82]:
np.mean(res['test_score'])

0.6281333333333333

## Cross validate 

In [86]:
transactions_train

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.462997
1,33172,6,35,45.016998
2,33172,8,11,13.887000
3,33172,9,11,15.983000
4,33172,10,11,21.341000
...,...,...,...,...
26450572,43300,727,25,7.602000
26450573,43300,727,15,3.709000
26450574,43300,727,1,6.448000
26450575,43300,727,11,24.669001


In [87]:
group

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4
...,...,...
199,Услуги починки (металлические изделия),199
200,Телеграф,200
201,Мануальные терапевты,201
202,Скорая помощь,202


In [20]:
from sklearn.multiclass import OneVsRestClassifier
import eli5

Using TensorFlow backend.


In [22]:
# def preproc_params(dct):
#     dct['max_depth'] = int(round(dct['max_depth']))
#     dct['min_data_in_leaf'] = int(round(dct['min_data_in_leaf']))
#     dct['num_leaves'] = int(round(dct['num_leaves']))
    
#     return dct

In [23]:
# lgb_params = optimizer.max['params']
# lgb_params = preproc_params(lgb_params)

In [24]:
# lgb_params = optimizer.max['params']
# lgb_params = preproc_params(lgb_params)

# lgb_params = {'bagging_fraction': 0.64183588863209,
#  'feature_fraction': 0.8517663630293925,
#  'learning_rate': 0.026527375714140905,
#  'max_depth': 15,
#  'min_child_weight': 0.45025619488575386,
#  'min_data_in_leaf': 222,
#  'num_leaves': 132,
#  'reg_alpha': 1.6624228537423238,
#  'reg_lambda': 5.4972043534631}

# model = lgb.LGBMClassifier(earning_rate=0.025, n_estimators=2000, num_laeves=69,reg_lambda=11, random_seed=1007)
# model = lgb.LGBMClassifier(**lgb_params, n_estimators=2000, random_state=47)

In [22]:
# model = cb.CatBoostClassifier(iterations = 2000, learning_rate = 0.04, depth = 5,
#                               loss_function = 'MultiClass', eval_metric='Accuracy',
#                               random_state = 53, task_type="GPU", devices='0', verbose = 1)


## Cross_val 

In [83]:
model = cb.CatBoostClassifier(iterations = 5000, learning_rate = 0.04, depth = 5,
                              loss_function = 'MultiClass', eval_metric='Accuracy',
                              random_state = 53, task_type="GPU", devices='0', verbose = 1)



In [84]:
res = cross_validate(model, train_data, train_target, cv=3, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.5168500	total: 38.1ms	remaining: 3m 10s
1:	learn: 0.5320000	total: 76.4ms	remaining: 3m 10s
2:	learn: 0.5353000	total: 111ms	remaining: 3m 4s
3:	learn: 0.5355500	total: 142ms	remaining: 2m 57s
4:	learn: 0.5405500	total: 173ms	remaining: 2m 53s
5:	learn: 0.5401000	total: 209ms	remaining: 2m 54s
6:	learn: 0.5435500	total: 244ms	remaining: 2m 53s
7:	learn: 0.5456000	total: 279ms	remaining: 2m 54s
8:	learn: 0.5450000	total: 314ms	remaining: 2m 54s
9:	learn: 0.5492500	total: 347ms	remaining: 2m 52s
10:	learn: 0.5503500	total: 378ms	remaining: 2m 51s
11:	learn: 0.5517500	total: 411ms	remaining: 2m 51s
12:	learn: 0.5513000	total: 443ms	remaining: 2m 49s
13:	learn: 0.5518000	total: 477ms	remaining: 2m 50s
14:	learn: 0.5529000	total: 506ms	remaining: 2m 48s
15:	learn: 0.5551000	total: 536ms	remaining: 2m 47s
16:	learn: 0.5568000	total: 564ms	remaining: 2m 45s
17:	learn: 0.5584500	total: 591ms	remaining: 2m 43s
18:	learn: 0.5576500	total: 616ms	remaining: 2m 41s
19:	learn: 0.5587500	

164:	learn: 0.6286000	total: 4.44s	remaining: 2m 10s
165:	learn: 0.6292000	total: 4.47s	remaining: 2m 10s
166:	learn: 0.6294500	total: 4.5s	remaining: 2m 10s
167:	learn: 0.6301000	total: 4.52s	remaining: 2m 10s
168:	learn: 0.6308000	total: 4.55s	remaining: 2m 9s
169:	learn: 0.6309500	total: 4.57s	remaining: 2m 9s
170:	learn: 0.6309500	total: 4.6s	remaining: 2m 9s
171:	learn: 0.6312500	total: 4.63s	remaining: 2m 9s
172:	learn: 0.6316500	total: 4.65s	remaining: 2m 9s
173:	learn: 0.6318000	total: 4.67s	remaining: 2m 9s
174:	learn: 0.6325500	total: 4.7s	remaining: 2m 9s
175:	learn: 0.6325000	total: 4.72s	remaining: 2m 9s
176:	learn: 0.6327000	total: 4.75s	remaining: 2m 9s
177:	learn: 0.6327500	total: 4.78s	remaining: 2m 9s
178:	learn: 0.6329500	total: 4.8s	remaining: 2m 9s
179:	learn: 0.6331500	total: 4.82s	remaining: 2m 9s
180:	learn: 0.6333500	total: 4.85s	remaining: 2m 9s
181:	learn: 0.6338000	total: 4.87s	remaining: 2m 8s
182:	learn: 0.6337500	total: 4.89s	remaining: 2m 8s
183:	learn: 

323:	learn: 0.6620500	total: 8.52s	remaining: 2m 2s
324:	learn: 0.6625000	total: 8.54s	remaining: 2m 2s
325:	learn: 0.6627000	total: 8.56s	remaining: 2m 2s
326:	learn: 0.6627000	total: 8.59s	remaining: 2m 2s
327:	learn: 0.6629000	total: 8.61s	remaining: 2m 2s
328:	learn: 0.6628500	total: 8.64s	remaining: 2m 2s
329:	learn: 0.6627500	total: 8.66s	remaining: 2m 2s
330:	learn: 0.6629500	total: 8.69s	remaining: 2m 2s
331:	learn: 0.6633000	total: 8.71s	remaining: 2m 2s
332:	learn: 0.6632500	total: 8.73s	remaining: 2m 2s
333:	learn: 0.6636500	total: 8.76s	remaining: 2m 2s
334:	learn: 0.6640500	total: 8.78s	remaining: 2m 2s
335:	learn: 0.6640500	total: 8.8s	remaining: 2m 2s
336:	learn: 0.6639500	total: 8.83s	remaining: 2m 2s
337:	learn: 0.6645500	total: 8.86s	remaining: 2m 2s
338:	learn: 0.6645000	total: 8.88s	remaining: 2m 2s
339:	learn: 0.6652000	total: 8.9s	remaining: 2m 2s
340:	learn: 0.6658500	total: 8.93s	remaining: 2m 2s
341:	learn: 0.6660500	total: 8.96s	remaining: 2m 2s
342:	learn: 0.

485:	learn: 0.6883500	total: 12.3s	remaining: 1m 54s
486:	learn: 0.6881500	total: 12.4s	remaining: 1m 54s
487:	learn: 0.6887500	total: 12.4s	remaining: 1m 54s
488:	learn: 0.6890000	total: 12.4s	remaining: 1m 54s
489:	learn: 0.6894500	total: 12.4s	remaining: 1m 54s
490:	learn: 0.6893500	total: 12.5s	remaining: 1m 54s
491:	learn: 0.6897500	total: 12.5s	remaining: 1m 54s
492:	learn: 0.6895000	total: 12.5s	remaining: 1m 54s
493:	learn: 0.6899500	total: 12.5s	remaining: 1m 54s
494:	learn: 0.6899500	total: 12.6s	remaining: 1m 54s
495:	learn: 0.6899000	total: 12.6s	remaining: 1m 54s
496:	learn: 0.6898500	total: 12.6s	remaining: 1m 54s
497:	learn: 0.6903500	total: 12.6s	remaining: 1m 54s
498:	learn: 0.6906000	total: 12.6s	remaining: 1m 54s
499:	learn: 0.6910000	total: 12.7s	remaining: 1m 54s
500:	learn: 0.6911000	total: 12.7s	remaining: 1m 54s
501:	learn: 0.6912500	total: 12.7s	remaining: 1m 54s
502:	learn: 0.6917000	total: 12.7s	remaining: 1m 53s
503:	learn: 0.6919500	total: 12.8s	remaining: 

...

4469:	learn: 0.9317000	total: 1m 38s	remaining: 11.7s
4470:	learn: 0.9319000	total: 1m 38s	remaining: 11.7s
4471:	learn: 0.9318500	total: 1m 38s	remaining: 11.6s
4472:	learn: 0.9318500	total: 1m 38s	remaining: 11.6s
4473:	learn: 0.9319500	total: 1m 38s	remaining: 11.6s
4474:	learn: 0.9319000	total: 1m 38s	remaining: 11.6s
4475:	learn: 0.9318500	total: 1m 38s	remaining: 11.6s
4476:	learn: 0.9318000	total: 1m 38s	remaining: 11.5s
4477:	learn: 0.9318000	total: 1m 38s	remaining: 11.5s
4478:	learn: 0.9317500	total: 1m 38s	remaining: 11.5s
4479:	learn: 0.9317000	total: 1m 38s	remaining: 11.5s
4480:	learn: 0.9319000	total: 1m 38s	remaining: 11.4s
4481:	learn: 0.9319000	total: 1m 38s	remaining: 11.4s
4482:	learn: 0.9320500	total: 1m 38s	remaining: 11.4s
4483:	learn: 0.9320000	total: 1m 38s	remaining: 11.4s
4484:	learn: 0.9319500	total: 1m 38s	remaining: 11.4s
4485:	learn: 0.9319500	total: 1m 38s	remaining: 11.3s
4486:	learn: 0.9319500	total: 1m 38s	remaining: 11.3s
4487:	learn: 0.9321000	total

4630:	learn: 0.9349500	total: 1m 41s	remaining: 8.12s
4631:	learn: 0.9349500	total: 1m 41s	remaining: 8.1s
4632:	learn: 0.9350000	total: 1m 41s	remaining: 8.08s
4633:	learn: 0.9349500	total: 1m 42s	remaining: 8.06s
4634:	learn: 0.9350000	total: 1m 42s	remaining: 8.03s
4635:	learn: 0.9350500	total: 1m 42s	remaining: 8.01s
4636:	learn: 0.9352000	total: 1m 42s	remaining: 7.99s
4637:	learn: 0.9352500	total: 1m 42s	remaining: 7.97s
4638:	learn: 0.9352500	total: 1m 42s	remaining: 7.95s
4639:	learn: 0.9352500	total: 1m 42s	remaining: 7.92s
4640:	learn: 0.9353000	total: 1m 42s	remaining: 7.9s
4641:	learn: 0.9354000	total: 1m 42s	remaining: 7.88s
4642:	learn: 0.9353500	total: 1m 42s	remaining: 7.86s
4643:	learn: 0.9353500	total: 1m 42s	remaining: 7.83s
4644:	learn: 0.9355500	total: 1m 42s	remaining: 7.81s
4645:	learn: 0.9354000	total: 1m 42s	remaining: 7.79s
4646:	learn: 0.9355500	total: 1m 42s	remaining: 7.77s
4647:	learn: 0.9355500	total: 1m 42s	remaining: 7.75s
4648:	learn: 0.9355500	total: 

4789:	learn: 0.9393500	total: 1m 45s	remaining: 4.61s
4790:	learn: 0.9395000	total: 1m 45s	remaining: 4.59s
4791:	learn: 0.9392500	total: 1m 45s	remaining: 4.57s
4792:	learn: 0.9393000	total: 1m 45s	remaining: 4.55s
4793:	learn: 0.9392500	total: 1m 45s	remaining: 4.53s
4794:	learn: 0.9393500	total: 1m 45s	remaining: 4.5s
4795:	learn: 0.9392500	total: 1m 45s	remaining: 4.48s
4796:	learn: 0.9392500	total: 1m 45s	remaining: 4.46s
4797:	learn: 0.9392500	total: 1m 45s	remaining: 4.44s
4798:	learn: 0.9392500	total: 1m 45s	remaining: 4.42s
4799:	learn: 0.9391000	total: 1m 45s	remaining: 4.39s
4800:	learn: 0.9392000	total: 1m 45s	remaining: 4.37s
4801:	learn: 0.9393000	total: 1m 45s	remaining: 4.35s
4802:	learn: 0.9395000	total: 1m 45s	remaining: 4.33s
4803:	learn: 0.9392500	total: 1m 45s	remaining: 4.31s
4804:	learn: 0.9393000	total: 1m 45s	remaining: 4.28s
4805:	learn: 0.9394500	total: 1m 45s	remaining: 4.26s
4806:	learn: 0.9393500	total: 1m 45s	remaining: 4.24s
4807:	learn: 0.9395000	total:

4948:	learn: 0.9428000	total: 1m 48s	remaining: 1.12s
4949:	learn: 0.9427500	total: 1m 48s	remaining: 1.1s
4950:	learn: 0.9427000	total: 1m 48s	remaining: 1.07s
4951:	learn: 0.9426500	total: 1m 48s	remaining: 1.05s
4952:	learn: 0.9426500	total: 1m 48s	remaining: 1.03s
4953:	learn: 0.9427000	total: 1m 48s	remaining: 1.01s
4954:	learn: 0.9427000	total: 1m 48s	remaining: 988ms
4955:	learn: 0.9427000	total: 1m 48s	remaining: 966ms
4956:	learn: 0.9427000	total: 1m 48s	remaining: 944ms
4957:	learn: 0.9427500	total: 1m 48s	remaining: 922ms
4958:	learn: 0.9427500	total: 1m 48s	remaining: 900ms
4959:	learn: 0.9428000	total: 1m 48s	remaining: 878ms
4960:	learn: 0.9428500	total: 1m 48s	remaining: 856ms
4961:	learn: 0.9430500	total: 1m 48s	remaining: 834ms
4962:	learn: 0.9429000	total: 1m 48s	remaining: 812ms
4963:	learn: 0.9429500	total: 1m 48s	remaining: 790ms
4964:	learn: 0.9431000	total: 1m 48s	remaining: 768ms
4965:	learn: 0.9431000	total: 1m 48s	remaining: 746ms
4966:	learn: 0.9430000	total:

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.6min finished


In [85]:
np.mean(res['test_score'])

0.6337333333333334

#0,630

## Param search 

In [ ]:
cat_features = [f'max_trans_group_day_{i}' for i in range]
max_trans_date
max_trans_group

In [32]:
list(train_data.columns)

['max_trans_group_day_480',
 'day_203_sum',
 'max_trans_group_day_206',
 'day_567_sum',
 'day_718_sum',
 'day_350_sum',
 'day_570_sum',
 'day_309_sum',
 'max_trans_group_day_626',
 'day_34_sum',
 'max_trans_group_day_133',
 'max_trans_group_day_369',
 'max_trans_group_day_34',
 'max_trans_group_day_174',
 'day_686_sum',
 'day_74_sum',
 'day_276_sum',
 'small_group_sum_33',
 'small_group_93',
 'day_27_sum',
 'max_trans_group_day_416',
 'small_group_136',
 'max_trans_group_day_283',
 'max_trans_group_day_351',
 'max_trans_group_day_214',
 'max_trans_group_day_219',
 'max_trans_group_day_181',
 'small_group_median_94',
 'max_trans_group_day_567',
 'max_trans_group_day_691',
 'small_group_std_187',
 'small_group_sum_153',
 'max_trans_group_day_332',
 'day_518_sum',
 'small_group_sum_182',
 'small_group_std_47',
 'max_trans_group_day_465',
 'max_trans_group_day_301',
 'small_group_median_76',
 'small_group_std_191',
 'max_trans_group_day_676',
 'max_trans_group_day_630',
 'day_425_sum',
 'd

In [ ]:
def get_cat_index(dataframe):
    cat_features = ['player_team','winner_team','hero_id','hero_pick_order','leaver_status','party_players','winner', 'abil_1',
 'abil_2',
 'abil_3',
 'abil_4',
 'abil_5',
 'abil_6',
 'abil_7',
 'abil_8',
 'abil_9',
 'abil_10',
 'abil_11',
 'abil_12',
 'abil_13',
 'abil_14',
 'abil_15',
 'abil_16',
 'abil_17',
 'abil_18',
 'abil_19',
 'abil_20',
 'abil_21',
 'abil_22',
 'abil_23',
 'abil_24',
 'abil_25',
 'first_blood_claimed','']
    categorical_features_index = [i for i, j in  enumerate(dataframe.columns) if j in cat_features]
    
    return categorical_features_index

In [ ]:
from paramsearch import paramsearch

In [ ]:
loss_function = 'MultiClass', eval_metric='Accuracy',
                              random_state = 53, task_type="GPU", devices='0', verbose = 1

In [ ]:
params = {'depth':[3,1,2,6,4,5,7,8,9,10,11,12,13],
          'iterations':[250,100,500,1000, 2000, 5000],
          'learning_rate':[0.04, 0.03, 0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'thread_count':8,
          'loss_function':'MultiClass',
          'eval_metric' : 'Accuracy',
          'task_type':'GPU',
          'devices':'0'
         'verbose':False}
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,train.drop(to_drop,axis=1),y_train,cat_feat)

## Predict 

In [33]:
model.fit(train_data, train_target, verbose=1)

0:	learn: 0.5035333	total: 34.4ms	remaining: 1m 8s
1:	learn: 0.5194667	total: 65.2ms	remaining: 1m 5s
2:	learn: 0.5226333	total: 95.9ms	remaining: 1m 3s
3:	learn: 0.5235000	total: 125ms	remaining: 1m 2s
4:	learn: 0.5318000	total: 154ms	remaining: 1m 1s
5:	learn: 0.5377000	total: 183ms	remaining: 1m
6:	learn: 0.5387333	total: 211ms	remaining: 1m
7:	learn: 0.5440667	total: 238ms	remaining: 59.2s
8:	learn: 0.5441000	total: 266ms	remaining: 58.8s
9:	learn: 0.5437000	total: 293ms	remaining: 58.3s
10:	learn: 0.5440000	total: 319ms	remaining: 57.7s
11:	learn: 0.5473333	total: 344ms	remaining: 57s
12:	learn: 0.5481000	total: 371ms	remaining: 56.6s
13:	learn: 0.5502333	total: 396ms	remaining: 56.2s
14:	learn: 0.5516000	total: 423ms	remaining: 55.9s
15:	learn: 0.5532667	total: 449ms	remaining: 55.7s
16:	learn: 0.5548333	total: 475ms	remaining: 55.5s
17:	learn: 0.5554000	total: 501ms	remaining: 55.2s
18:	learn: 0.5571000	total: 528ms	remaining: 55.1s
19:	learn: 0.5577333	total: 554ms	remaining: 5

161:	learn: 0.6230667	total: 4.2s	remaining: 48.6s
162:	learn: 0.6237000	total: 4.23s	remaining: 48.5s
163:	learn: 0.6238667	total: 4.25s	remaining: 48.5s
164:	learn: 0.6239000	total: 4.28s	remaining: 48.5s
165:	learn: 0.6244667	total: 4.3s	remaining: 48.4s
166:	learn: 0.6247333	total: 4.36s	remaining: 48.7s
167:	learn: 0.6251000	total: 4.38s	remaining: 48.7s
168:	learn: 0.6251000	total: 4.41s	remaining: 48.7s
169:	learn: 0.6254000	total: 4.41s	remaining: 48.7s
170:	learn: 0.6260333	total: 4.43s	remaining: 48.6s
171:	learn: 0.6260667	total: 4.46s	remaining: 48.5s
172:	learn: 0.6262000	total: 4.48s	remaining: 48.5s
173:	learn: 0.6265333	total: 4.54s	remaining: 48.7s
174:	learn: 0.6260000	total: 4.56s	remaining: 48.7s
175:	learn: 0.6264000	total: 4.59s	remaining: 48.6s
176:	learn: 0.6266333	total: 4.61s	remaining: 48.6s
177:	learn: 0.6267000	total: 4.61s	remaining: 48.6s
178:	learn: 0.6270667	total: 4.64s	remaining: 48.5s
179:	learn: 0.6274667	total: 4.66s	remaining: 48.5s
180:	learn: 0.

322:	learn: 0.6509000	total: 8.35s	remaining: 44.3s
323:	learn: 0.6509000	total: 8.37s	remaining: 44.3s
324:	learn: 0.6510333	total: 8.39s	remaining: 44.2s
325:	learn: 0.6511667	total: 8.41s	remaining: 44.2s
326:	learn: 0.6514333	total: 8.44s	remaining: 44.1s
327:	learn: 0.6512000	total: 8.46s	remaining: 44.1s
328:	learn: 0.6516000	total: 8.48s	remaining: 44s
329:	learn: 0.6516667	total: 8.51s	remaining: 44s
330:	learn: 0.6516667	total: 8.54s	remaining: 44s
331:	learn: 0.6519333	total: 8.56s	remaining: 43.9s
332:	learn: 0.6522333	total: 8.58s	remaining: 43.9s
333:	learn: 0.6523333	total: 8.61s	remaining: 43.9s
334:	learn: 0.6522000	total: 8.64s	remaining: 43.8s
335:	learn: 0.6524000	total: 8.66s	remaining: 43.8s
336:	learn: 0.6524667	total: 8.69s	remaining: 43.8s
337:	learn: 0.6530333	total: 8.71s	remaining: 43.7s
338:	learn: 0.6530667	total: 8.73s	remaining: 43.7s
339:	learn: 0.6531333	total: 8.76s	remaining: 43.7s
340:	learn: 0.6538000	total: 8.78s	remaining: 43.6s
341:	learn: 0.6539

488:	learn: 0.6706333	total: 12.5s	remaining: 39.3s
489:	learn: 0.6709333	total: 12.5s	remaining: 39.2s
490:	learn: 0.6710333	total: 12.5s	remaining: 39.2s
491:	learn: 0.6712333	total: 12.6s	remaining: 39.2s
492:	learn: 0.6712000	total: 12.6s	remaining: 39.1s
493:	learn: 0.6716000	total: 12.6s	remaining: 39.1s
494:	learn: 0.6714000	total: 12.6s	remaining: 39.1s
495:	learn: 0.6714000	total: 12.6s	remaining: 39s
496:	learn: 0.6719000	total: 12.7s	remaining: 39s
497:	learn: 0.6718000	total: 12.7s	remaining: 39s
498:	learn: 0.6719667	total: 12.7s	remaining: 38.9s
499:	learn: 0.6722000	total: 12.7s	remaining: 38.9s
500:	learn: 0.6721667	total: 12.8s	remaining: 38.9s
501:	learn: 0.6722000	total: 12.8s	remaining: 38.8s
502:	learn: 0.6725000	total: 12.8s	remaining: 38.8s
503:	learn: 0.6727000	total: 12.8s	remaining: 38.8s
504:	learn: 0.6728000	total: 12.9s	remaining: 38.7s
505:	learn: 0.6725667	total: 12.9s	remaining: 38.7s
506:	learn: 0.6728667	total: 12.9s	remaining: 38.7s
507:	learn: 0.6728

655:	learn: 0.6872667	total: 16.7s	remaining: 34.9s
656:	learn: 0.6877667	total: 16.8s	remaining: 34.9s
657:	learn: 0.6876667	total: 16.8s	remaining: 34.9s
658:	learn: 0.6876667	total: 16.8s	remaining: 34.8s
659:	learn: 0.6879333	total: 16.8s	remaining: 34.8s
660:	learn: 0.6881333	total: 16.8s	remaining: 34.8s
661:	learn: 0.6884333	total: 16.8s	remaining: 34.7s
662:	learn: 0.6883667	total: 16.9s	remaining: 34.7s
663:	learn: 0.6884333	total: 16.9s	remaining: 34.7s
664:	learn: 0.6884000	total: 16.9s	remaining: 34.6s
665:	learn: 0.6885333	total: 17s	remaining: 34.7s
666:	learn: 0.6889667	total: 17s	remaining: 34.7s
667:	learn: 0.6888667	total: 17s	remaining: 34.6s
668:	learn: 0.6886000	total: 17.1s	remaining: 34.6s
669:	learn: 0.6889000	total: 17.1s	remaining: 34.6s
670:	learn: 0.6890333	total: 17.1s	remaining: 34.5s
671:	learn: 0.6891333	total: 17.1s	remaining: 34.5s
672:	learn: 0.6892000	total: 17.2s	remaining: 34.5s
673:	learn: 0.6895000	total: 17.2s	remaining: 34.5s
674:	learn: 0.6897

...

1307:	learn: 0.7401333	total: 32.1s	remaining: 17.2s
1308:	learn: 0.7401667	total: 32.1s	remaining: 17.1s
1309:	learn: 0.7404000	total: 32.1s	remaining: 17.1s
1310:	learn: 0.7403667	total: 32.1s	remaining: 17.1s
1311:	learn: 0.7404000	total: 32.2s	remaining: 17s
1312:	learn: 0.7405000	total: 32.2s	remaining: 17s
1313:	learn: 0.7403000	total: 32.2s	remaining: 17s
1314:	learn: 0.7404000	total: 32.2s	remaining: 17s
1315:	learn: 0.7405333	total: 32.3s	remaining: 16.9s
1316:	learn: 0.7406000	total: 32.3s	remaining: 16.9s
1317:	learn: 0.7408333	total: 32.3s	remaining: 16.9s
1318:	learn: 0.7407333	total: 32.3s	remaining: 16.9s
1319:	learn: 0.7409667	total: 32.4s	remaining: 16.8s
1320:	learn: 0.7409333	total: 32.4s	remaining: 16.8s
1321:	learn: 0.7411000	total: 32.4s	remaining: 16.8s
1322:	learn: 0.7411667	total: 32.4s	remaining: 16.8s
1323:	learn: 0.7413333	total: 32.5s	remaining: 16.7s
1324:	learn: 0.7414000	total: 32.5s	remaining: 16.7s
1325:	learn: 0.7415333	total: 32.5s	remaining: 16.7s
1

1463:	learn: 0.7509333	total: 35.7s	remaining: 13.2s
1464:	learn: 0.7508667	total: 35.7s	remaining: 13.2s
1465:	learn: 0.7509667	total: 35.7s	remaining: 13.1s
1466:	learn: 0.7512000	total: 35.7s	remaining: 13.1s
1467:	learn: 0.7511000	total: 35.7s	remaining: 13.1s
1468:	learn: 0.7512000	total: 35.8s	remaining: 13.1s
1469:	learn: 0.7509667	total: 35.8s	remaining: 13s
1470:	learn: 0.7510333	total: 35.8s	remaining: 13s
1471:	learn: 0.7511667	total: 35.8s	remaining: 13s
1472:	learn: 0.7514333	total: 35.9s	remaining: 13s
1473:	learn: 0.7515333	total: 35.9s	remaining: 12.9s
1474:	learn: 0.7514333	total: 35.9s	remaining: 12.9s
1475:	learn: 0.7516333	total: 35.9s	remaining: 12.9s
1476:	learn: 0.7514667	total: 36s	remaining: 12.9s
1477:	learn: 0.7516333	total: 36s	remaining: 12.8s
1478:	learn: 0.7517000	total: 36s	remaining: 12.8s
1479:	learn: 0.7516667	total: 36s	remaining: 12.8s
1480:	learn: 0.7517667	total: 36.1s	remaining: 12.8s
1481:	learn: 0.7516333	total: 36.1s	remaining: 12.7s
1482:	lea

1624:	learn: 0.7598000	total: 39.5s	remaining: 9.19s
1625:	learn: 0.7598000	total: 39.5s	remaining: 9.16s
1626:	learn: 0.7598333	total: 39.5s	remaining: 9.14s
1627:	learn: 0.7600000	total: 39.5s	remaining: 9.11s
1628:	learn: 0.7600000	total: 39.5s	remaining: 9.09s
1629:	learn: 0.7599333	total: 39.6s	remaining: 9.06s
1630:	learn: 0.7599000	total: 39.6s	remaining: 9.04s
1631:	learn: 0.7600333	total: 39.6s	remaining: 9.02s
1632:	learn: 0.7601667	total: 39.6s	remaining: 8.99s
1633:	learn: 0.7603000	total: 39.7s	remaining: 8.97s
1634:	learn: 0.7604000	total: 39.7s	remaining: 8.94s
1635:	learn: 0.7604000	total: 39.7s	remaining: 8.92s
1636:	learn: 0.7602667	total: 39.7s	remaining: 8.89s
1637:	learn: 0.7603667	total: 39.8s	remaining: 8.87s
1638:	learn: 0.7603000	total: 39.8s	remaining: 8.84s
1639:	learn: 0.7604000	total: 39.8s	remaining: 8.82s
1640:	learn: 0.7604000	total: 39.8s	remaining: 8.79s
1641:	learn: 0.7604333	total: 39.9s	remaining: 8.77s
1642:	learn: 0.7602667	total: 39.9s	remaining:

1788:	learn: 0.7689667	total: 43.2s	remaining: 5.14s
1789:	learn: 0.7688667	total: 43.3s	remaining: 5.12s
1790:	learn: 0.7688667	total: 43.3s	remaining: 5.09s
1791:	learn: 0.7693000	total: 43.3s	remaining: 5.07s
1792:	learn: 0.7693333	total: 43.3s	remaining: 5.05s
1793:	learn: 0.7691333	total: 43.3s	remaining: 5.02s
1794:	learn: 0.7691667	total: 43.4s	remaining: 5s
1795:	learn: 0.7693333	total: 43.4s	remaining: 4.97s
1796:	learn: 0.7695667	total: 43.4s	remaining: 4.95s
1797:	learn: 0.7695333	total: 43.4s	remaining: 4.92s
1798:	learn: 0.7691667	total: 43.5s	remaining: 4.9s
1799:	learn: 0.7696333	total: 43.5s	remaining: 4.87s
1800:	learn: 0.7698333	total: 43.5s	remaining: 4.85s
1801:	learn: 0.7696333	total: 43.5s	remaining: 4.83s
1802:	learn: 0.7694333	total: 43.5s	remaining: 4.8s
1803:	learn: 0.7693667	total: 43.6s	remaining: 4.78s
1804:	learn: 0.7696000	total: 43.6s	remaining: 4.75s
1805:	learn: 0.7694333	total: 43.6s	remaining: 4.73s
1806:	learn: 0.7694667	total: 43.6s	remaining: 4.7s

1945:	learn: 0.7774000	total: 46.8s	remaining: 1.31s
1946:	learn: 0.7776667	total: 46.8s	remaining: 1.28s
1947:	learn: 0.7777667	total: 46.8s	remaining: 1.26s
1948:	learn: 0.7777667	total: 46.8s	remaining: 1.24s
1949:	learn: 0.7778667	total: 46.9s	remaining: 1.21s
1950:	learn: 0.7779667	total: 46.9s	remaining: 1.19s
1951:	learn: 0.7781667	total: 46.9s	remaining: 1.16s
1952:	learn: 0.7783000	total: 46.9s	remaining: 1.14s
1953:	learn: 0.7785333	total: 46.9s	remaining: 1.11s
1954:	learn: 0.7785667	total: 47s	remaining: 1.09s
1955:	learn: 0.7786333	total: 47s	remaining: 1.06s
1956:	learn: 0.7785667	total: 47s	remaining: 1.04s
1957:	learn: 0.7786000	total: 47s	remaining: 1.02s
1958:	learn: 0.7786667	total: 47.1s	remaining: 993ms
1959:	learn: 0.7785000	total: 47.1s	remaining: 969ms
1960:	learn: 0.7786333	total: 47.1s	remaining: 944ms
1961:	learn: 0.7787667	total: 47.1s	remaining: 920ms
1962:	learn: 0.7789333	total: 47.1s	remaining: 896ms
1963:	learn: 0.7787000	total: 47.2s	remaining: 872ms
1

In [34]:
eli5.catboost.explain_weights(model, top=1000)

Weight,Feature
0.0379,small_group_sum_1
0.0364,small_group_median_1
0.0268,small_group_12
0.0259,small_group_11
0.0167,small_group_sum_3
0.0164,small_group_sum_11
0.0137,small_group_sum_4
0.0132,small_group_sum_57
0.0119,small_group_median_25
0.0102,small_group_sum_61


In [69]:
pred=model.predict(X_test)

In [71]:
pred.reshape(-1)

array([0., 2., 3., ..., 0., 2., 0.])

In [74]:
submission = pd.DataFrame({'bins': pred.reshape(-1)}, index=test.client_id)

In [76]:
submission.to_csv('st.csv')